In [ ]:
import torch, math
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision.datasets as dsets
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F
torch.__version__

## Fashion MNIST 进行分类
### Fashion MNIST介绍

Fashion MNIST数据集是kaggle上提供的一个图像分类入门级的数据集，其中包含10个类别的70000个灰度图像。如图所示，这些图片显示的是每件衣服的低分辨率(28×28像素)。`Fashion MNIST`的目标是作为经典MNIST数据的替换——通常被用作计算机视觉机器学习程序的“Hello, World”。

### 数据集介绍
#### 分类

- 0 T-shirt/top
- 1 Trouser
- 2 Pullover
- 3 Dress
- 4 Coat
- 5 Sandal
- 6 Shirt
- 7 Sneaker
- 8 Bag
- 9 Ankle boot 

#### 格式
存储的训练的数据和测试的数据，格式如下：

label是分类的标签 pixel1-pixel784是每一个像素代表的值。因为是灰度图像，所以是一个0-255之间的数值。

为什么是784个像素？ 28 * 28 = 784

#### 数据提交
Fashion MNIST不需要我们进行数据的提交，数据集中已经帮助我们将 训练集和测试集分好了，我们只需要载入、训练、查看即可，所以Fashion MNIST 是一个非常好的入门级别的数据集。





In [ ]:
DATA_PATH = Path('./data/')

In [ ]:
train = pd.read_csv(DATA_PATH / "fashion-minist_train.csv")
train.head(10)

In [ ]:
test = pd.read_csv(DATA_PATH / "fashion-minist_test.csv")
test.head(10)

In [ ]:
train.max()

In [ ]:
import struct
from PIL import Image

with open(DATA_PATH / "train-images-idx3-ubyte", "rb") as file_object:
    header_data = struct.unpack(">4I", file_object.read(16))
    print(header_data)

In [ ]:
with open(DATA_PATH / "train-labels-idx1-ubyte", "rb") as file_object:
    header_data = struct.unpack(">2I", file_object.read(8))
    print(header_data)

有四字节的`header_data`，故使用`unpack_from`进行二进制转换时，偏置`offset=16`

In [ ]:
with open(DATA_PATH / "train-images-idx3-ubyte", "rb") as file_object:
    raw_img = file_object.read()

img = struct.unpack_from(">784B", raw_img, 16)
image = np.asarray(img).reshape(28, 28)
print(image.shape)
plt.imshow(image, cmap = plt.cm.gray)
plt.show()

In [ ]:
with open(DATA_PATH / "train-labels-idx1-ubyte", "rb") as file_object:
    raw_img = file_object.read(1)
    label = struct.unpack(">B", raw_img)
    print(label)

### 数据加载
为了使用`pytorch`的`dataloader`进行数据的加载，需要先创建一个自定义的`dataset`

In [ ]:
class FashionMNISTDataset(Dataset):
    def __init__(self, csv_file, transform = None):
        data = pd.read_csv(csv_file)
        self.X = np.array(data.iloc[:, 1:]).reshape(-1, 1, 28, 28).astype(float)
        self.Y = np.array(data.iloc[:, 0]);

        del data; # 释放内存
        self.len = len(self.X)
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        item = self.X[idx]
        label = self.Y[idx]
        return (item, label)

对于自定义的数据集，只需要实现三个函数：

`__init__`:初始化函数主要用于数据的加载，这里直接使用`pandas`将数据读取为`dataframe`，然后将其转成`numpy`数组来进行索引。

`__len__`:返回数据集的总数，`pytorch`里面的`dataloader`需要知道数据集的总数

`__getitem__`:会返回单张图片，它包含一个`index`，返回值为样本及其标签


## 创建和训练测试集

In [ ]:
train_dataset = FashionMNISTDataset(csv_file = DATA_PATH / "fashion-mnist_train.csv")
test_dataset = FashionMNISTDataset(csv_file = DATA_PATH / "fashion-mnist_test.csv")

在使用Pytorch的DataLoader读取数据之前，需要指定一个batch size这也是一个超参数，涉及到内存的使用量，如果出现OOM的错误则要减小这个数值，一般这个数值都为2的幂或者2的倍数。

In [ ]:
#因为是常量，所以大写，需要说明的是，这些常量建议都使用完整的英文单词，减少歧义
BATCH_SIZE = 256 

我们接着使用`dataloader`模块来使用这些数据

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True)

In [ ]:
from random import shuffle


test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = False)

查看一下数据

In [ ]:
a = iter(train_loader)
data = next(a)
img = data[0][0].reshape(28, 28)
data[0][0].shape, img.shape

In [ ]:
plt.imshow(img, cmap = plt.cm.gray)
plt.show()

### 创建网络

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 5, padding = 2),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        self.pool1 = nn.MaxPool2d(2)

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size = 3),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.pool2 = nn.MaxPool2d(2)
        self.fc = nn.Linear(5 * 5 * 64, 10)

    
    def forward(self, x):
        out = self.layer1(x)
        out = self.pool1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.pool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
        

在函数里使用`torch.nn`提供的模块来定义各个层，在每个卷积层后使用了批次的归一化和RELU激活并且在每一个操作分组后面进行了pooling的操作（减少信息量，避免过拟合），后我们使用了全连接层来输出10个类别。

`view`函数用来改变输出值矩阵的形状来匹配最后一层的维度。

In [ ]:
cnn = CNN();
cnn(torch.rand(1, 1, 28, 28))

In [ ]:
print(cnn)

从定义模型开始就要指定模型计算的位置，CPU还是GPU，所以需要加另外一个参数

In [ ]:
DEVICE = torch.device("cpu")
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")

print(DEVICE)

In [ ]:
cnn = cnn.to(DEVICE)

### 损失函数
多分类因为使用`Softmax`回归将神经网络前向传播得到的结果变成概率分布 所以使用交叉熵损失。 

在`pytorch`中`NN.CrossEntropyLoss`是将`nn.LogSoftmax()`和`nn.NLLLoss()`进行了整合，`CrossEntropyLoss`,我们也可以分开来写使用两步计算，这里为了方便直接一步到位。

In [ ]:
#损失函数也需要放到GPU中
criterion = nn.CrossEntropyLoss().to(DEVICE)

### 优化器
Adam优化器

In [ ]:
LEARNING_RATE = 0.01 # 学习率
optimizer = torch.optim.Adam(cnn.parameters(), lr = LEARNING_RATE)

### 开始训练

In [ ]:
TOTAL_EPOCHS = 50

In [ ]:
%%time

losses = [];
for epoch in range(TOTAL_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.float().to(DEVICE)
        labels = labels.to(DEVICE)

        # 梯度清零
        optimizer.zero_grad()

        # 前向传播
        outputs = cnn(images)
        
        # 计算损失
        loss = criterion(outputs, labels)

        # 反向传播
        loss.backward()

        # 更新参数
        optimizer.step()

        # 记录损失
        losses.append(loss.cpu().data.item());

        if (i + 1) % 100 == 0:
            print('Epoch: %d%d, Iter: %d%d, Loss: %.4f' % (epoch + 1, TOTAL_EPOCHS, 
            i + 1, len(train_loader) // BATCH_SIZE, loss.data.item()))

### 可视化损失函数

In [ ]:
plt.xkcd();
plt.xlabel('Epoch #');
plt.ylabel('Loss');
plt.plot(losses);
plt.show()

### 保存模型

In [ ]:
torch.save(cnn.state_dict(), "fm-cnn3.pth")
# 加载用这个
#cnn.load_state_dict(torch.load("fm-cnn3.pth"))

### 模型评估
模型评估就是使用测试集对模型进行的评估，应该是添加到训练中进行了，这里为了方便说明直接在训练完成后评估了

In [ ]:
cnn.eval()
correct = 0
total = 0

for images, labels in test_loader:
    images = images.float().to(DEVICE)
    outputs = cnn(images).cpu()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy: %.4f %%' % (100 * correct / total))

模型评估的步骤如下：

1. 将网络的模式改为eval。
2. 将图片输入到网络中得到输出。
3. 通过取出one-hot输出的最大值来得到输出的 标签。
4. 统计正确的预测值。

### 进一步优化

In [ ]:
%%time
# 修改学习率和批次
cnn.train()
LEARNING_RATE = LEARNING_RATE * 0.1
TOTAL_EPOCHS = 20
optimizer = torch.optim.Adam(cnn.parameters(), lr = 0.001)
losses = [];

for epoch in range(TOTAL_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.float().to(DEVICE)
        labes = labels.to(DEVICE)

        # 梯度清零
        optimizer.zero_grad()
        outputs = cnn(images)

        # 计算损失
        loss = criterion(outputs, labels).cpu()
        loss.backward()
        optimizer.step()
        losses.append(loss.data.item())
        if (i + 1) % 100 == 0:
            print('Epoch: %d%d, Iter: %d%d, Loss: %.4f' % (epoch + 1, TOTAL_EPOCHS, 
            i + 1, len(train_loader) // BATCH_SIZE, loss.data.item()))

plt.xkcd();
plt.xlabel('Epoch #');
plt.ylabel('Loss');
plt.plot(losses);

In [ ]:
plt.show()

### 再次进行评估


In [ ]:
cnn.eval()
correct = 0
total = 0

for images, labels in test_loader:
    images = images.float().to(DEVICE)
    outputs = cnn(images).cpu()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy: %.4f %%' % (100 * correct / total))

In [ ]:
%%time
# 再次修改学习率和批次
cnn.train()
LEARNING_RATE = LEARNING_RATE * 0.1
TOTAL_EPOCHS = 10
optimizer = torch.optim.Adam(cnn.parameters(), lr = 0.001)
losses = [];

for epoch in range(TOTAL_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.float().to(DEVICE)
        labels = labels.to(DEVICE)

        # 梯度清零
        optimizer.zero_grad()
        outputs = cnn(images)

        # 计算损失
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.cpu().data.item());

        if (i + 1) % 100 == 0:
            print('Epoch: %d%d, Iter: %d%d, Loss: %.4f' % (epoch + 1, TOTAL_EPOCHS, 
            i + 1, len(train_loader) // BATCH_SIZE, loss.data.item()))


In [ ]:
plt.xkcd();
plt.xlabel('Epoch #');
plt.ylabel('Loss');
plt.plot(losses);
plt.show();

In [ ]:
cnn.eval()
correct = 0
total = 0
for images, labels in test_loader:
    images = images.float().to(DEVICE)
    outputs = cnn(images).cpu()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print('Accuracy: %.4f %%' % (100 * correct / total))

损失小了，但是准确率没有提高，这就说明已经接近模型的瓶颈了，如果再要进行优化，就需要修改模型了。另外还有一个判断模型是否到瓶颈的标准，就是看损失函数，最后一次的训练的损失函数明显的没有下降的趋势，只是在震荡，这说明已经没有什么优化的空间了。

通过简单的操作，我们也能够看到Adam优化器的暴力性，我们只要简单的修改学习率就能够达到优化的效果，Adam优化器的使用一般情况下是首先使用0.1进行预热，然后再用0.01进行大批次的训练，最后使用0.001这个学习率进行收尾，再小的学习率一般情况就不需要了。

## 总结
最后总结几个超参数:

`BATCH_SIZE`:批次数量，定义每次训练时多少数据作为一批，这个批次需要在dataloader初始化时进行设置，并且需要这对模型和显存进行配置，如果出现OOM有线减小，一般设为2的倍数

`DEVICE`:进行计算的设备，主要是CPU还是GPU

`LEARNING_RATE`:学习率，反向传播时使用

`TOTAL_EPOCHS`: 训练的批次，一般情况下会根据损失和准确率等阈值